# Movie Recommendation
The TMDb 5000 Movies dataset is a popular and widely used dataset in the field of data science and machine learning, particularly for tasks related to movie recommendation, analysis, and prediction. This dataset contains a wealth of information about thousands of movies, making it a valuable resource for various data-driven projects related to the film industry. 
We'll be using this in our project.

In [1]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
import base64
import io
import codecs

In [2]:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies=pd.read_csv('tmdb_5000_movies.csv')
movies=movies[[ 'genres', 'keywords','title', 'original_language', 'popularity', 'production_companies', 
        'vote_average',"id"]]

## Cleaning Of Data
The TMDb dataset contains information like genres, keywords, cast etc. in JSON format . To deal with them , we must convert them into strings. 

In [3]:
movies['genres'] = movies['genres'].apply(json.loads)
for index,i in zip(movies.index,movies['genres']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name'])) 
    movies.loc[index,'genres'] = str(list1)
movies['keywords'] = movies['keywords'].apply(json.loads)
for index,i in zip(movies.index,movies['keywords']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name'])) 
    movies.loc[index,'keywords'] = str(list1)
movies['production_companies'] = movies['production_companies'].apply(json.loads)
for index,i in zip(movies.index,movies['production_companies']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name']))
    movies.loc[index,'production_companies'] = str(list1)


In [4]:
credits['cast'] = credits['cast'].apply(json.loads)
for index,i in zip(credits.index,credits['cast']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name'])) 
    credits.loc[index,'cast'] = str(list1)
credits['crew'] = credits['crew'].apply(json.loads)
def director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
credits['crew'] = credits['crew'].apply(director)
credits.rename(columns={'crew':'director'},inplace=True)

We then merge the two datasets for further analysis.

In [5]:
movies=pd.merge(movies,credits,how="left",left_on='id', right_on='movie_id')
movies=movies[["id","title_x","genres","director","keywords","popularity","production_companies","vote_average","cast"]]

## Conversion to Binary Matrices
To apply kNNs, we first make a list of unique data and then represent it using logical matrices.

In [6]:
movies['genres'] = movies['genres'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
movies['genres'] = movies['genres'].str.split(',')
genreList = []
for index, row in movies.iterrows():
    genres = row["genres"]
    for genre in genres:
        if genre not in genreList:
            genreList.append(genre)
def binary(genre_list):
    binaryList = []
    
    for genre in genreList:
        if genre in genre_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList
movies['genres_bin'] = movies['genres'].apply(lambda x: binary(x))
movies['genres_bin'].head()

0    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: genres_bin, dtype: object

In [7]:
movies['cast'] = movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
movies['cast'] = movies['cast'].str.split(',')

In [8]:
for i,j in zip(movies['cast'],movies.index):
    list2 = []
    list2 = i[:4]
    movies.loc[j,'cast'] = str(list2)
movies['cast'] = movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['cast'] = movies['cast'].str.split(',')
for i,j in zip(movies['cast'],movies.index):
    list2 = []
    list2 = i
    list2.sort()
    movies.loc[j,'cast'] = str(list2)
movies['cast']=movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'')

In [9]:
castList = []
for index, row in movies.iterrows():
    cast = row["cast"]
    
    for i in cast:
        if i not in castList:
            castList.append(i)

In [10]:
def binary(cast_list):
    binaryList = []
    for genre in castList:
        if genre in cast_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList

In [11]:
movies['cast_bin'] = movies['cast'].apply(lambda x: binary(x))
movies['cast_bin']

0       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1       [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...
2       [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
3       [0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...
4       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...
                              ...                        
4798    [0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, ...
4799    [0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
4800    [0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...
4801    [0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, ...
4802    [0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, ...
Name: cast_bin, Length: 4803, dtype: object

In [12]:
def xstr(s):
    if s is None:
        return ''
    return str(s)
movies['director'] = movies['director'].apply(xstr)

In [13]:
directorList=[]
for i in movies['director']:
    if i not in directorList:
        directorList.append(i)
def binary(director_list):
    binaryList = []  
    for direct in directorList:
        if direct in director_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList
movies['director_bin'] = movies['director'].apply(lambda x: binary(x))
movies['director_bin'].head()

0    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: director_bin, dtype: object

In [14]:
movies['keywords'] = movies['keywords'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
movies['keywords'] = movies['keywords'].str.split(',')
for i,j in zip(movies['keywords'],movies.index):
    list2 = []
    list2 = i
    movies.loc[j,'keywords'] = str(list2)
movies['keywords'] = movies['keywords'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['keywords'] = movies['keywords'].str.split(',')
for i,j in zip(movies['keywords'],movies.index):
    list2 = []
    list2 = i
    list2.sort()
    movies.loc[j,'keywords'] = str(list2)
movies['keywords'] = movies['keywords'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['keywords'] = movies['keywords'].str.split(',')

In [15]:
words_list = []
for index, row in movies.iterrows():
    genres = row["keywords"]
    
    for genre in genres:
        if genre not in words_list:
            words_list.append(genre)
def binary(words):
    binaryList = []
    for genre in words_list:
        if genre in words:
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList
movies['words_bin'] = movies['keywords'].apply(lambda x: binary(x))
movies = movies[(movies['vote_average']!=0)] #removing the movies with 0 score and without drector names 
movies = movies[movies['director']!='']

## Similarity
We define the item-item similarity between movies in the following way . We apply <b>Cosine Similarity</b> using parameters - genres, cast,director and keywords.

In [16]:
from scipy import spatial

def Similarity(movieId1, movieId2):
    a = movies.iloc[movieId1]
    b = movies.iloc[movieId2]
    
    genresA = a['genres_bin']
    genresB = b['genres_bin']
    
    genreDistance = spatial.distance.cosine(genresA, genresB)
    
    scoreA = a['cast_bin']
    scoreB = b['cast_bin']
    scoreDistance = spatial.distance.cosine(scoreA, scoreB)
    
    directA = a['director_bin']
    directB = b['director_bin']
    directDistance = spatial.distance.cosine(directA, directB)
    
    wordsA = a['words_bin']
    wordsB = b['words_bin']
    wordsDistance = spatial.distance.cosine(directA, directB)
    return genreDistance + directDistance + scoreDistance + wordsDistance

In [17]:
new_id = list(range(0,movies.shape[0]))
movies['new_id']=new_id
movies=movies[['title_x','genres','vote_average','genres_bin','cast_bin','new_id','director','director_bin','words_bin']]
movies.head()

,title_x,genres,vote_average,genres_bin,cast_bin,new_id,director,director_bin,words_bin
0,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]",7.2,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,James Cameron,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...",1,Gore Verbinski,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Spectre,"[Action, Adventure, Crime]",6.3,"[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,Sam Mendes,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,"[1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...",3,Christopher Nolan,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,John Carter,"[Action, Adventure, ScienceFiction]",6.1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...",4,Andrew Stanton,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Score Prediction
We finally pass the similarity through a <b>Score Function</b> to give out 10 recommendations of movies.

In [18]:
import operator

def predict_score(name):
    #name = input('Enter a movie title: ')
    new_movie = movies[movies['title_x'].str.contains(name)].iloc[0].to_frame().T
    print('Selected Movie: ',new_movie.title_x.values[0])
    def getNeighbors(baseMovie, K):
        distances = []
    
        for index, movie in movies.iterrows():
            if movie['new_id'] != baseMovie['new_id'].values[0]:
                dist = Similarity(baseMovie['new_id'].values[0], movie['new_id'])
                distances.append((movie['new_id'], dist))
    
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
    
        for x in range(K):
            neighbors.append(distances[x])
        return neighbors

    K = 10
    avgRating = 0
    neighbors = getNeighbors(new_movie, K)
    
    print('\nRecommended Movies: \n')
    for neighbor in neighbors:
        avgRating = avgRating+movies.iloc[neighbor[0]][2]  
        print( movies.iloc[neighbor[0]][0]+" | Genres: "+str(movies.iloc[neighbor[0]][1]).strip('[]').replace(' ','')+" | Rating: "+str(movies.iloc[neighbor[0]][2]))
    
    print('\n')
    avgRating = avgRating/K
    print('The predicted rating for %s is: %f' %(new_movie['title_x'].values[0],avgRating))
    print('The actual rating for %s is %f' %(new_movie['title_x'].values[0],new_movie['vote_average']))


In [19]:
predict_score("Interstellar")

Selected Movie:  Interstellar

Recommended Movies: 

Inception | Genres: 'Action','Thriller','ScienceFiction','Mystery','Adventure' | Rating: 8.1
The Prestige | Genres: 'Drama','Mystery','Thriller' | Rating: 8.0
The Dark Knight Rises | Genres: 'Action','Crime','Drama','Thriller' | Rating: 7.6
The Dark Knight | Genres: 'Drama','Action','Crime','Thriller' | Rating: 8.2
Batman Begins | Genres: 'Action','Crime','Drama' | Rating: 7.5
Memento | Genres: 'Mystery','Thriller' | Rating: 8.1
The Martian | Genres: 'Drama','Adventure','ScienceFiction' | Rating: 7.6
Midnight Special | Genres: 'Adventure','Drama','ScienceFiction' | Rating: 6.2
Insomnia | Genres: 'Crime','Mystery','Thriller' | Rating: 6.8
20,000 Leagues Under the Sea | Genres: 'Adventure','Drama','ScienceFiction' | Rating: 6.8


The predicted rating for Interstellar is: 7.490000
The actual rating for Interstellar is 8.100000


In [20]:
predict_score("Love Actually")

Selected Movie:  Love Actually

Recommended Movies: 

The Boat That Rocked | Genres: 'Drama','Comedy' | Rating: 7.2
About Time | Genres: 'Comedy','Drama','ScienceFiction' | Rating: 7.8
Little Black Book | Genres: 'Comedy','Romance','Drama' | Rating: 5.2
Away We Go | Genres: 'Drama','Comedy','Romance' | Rating: 6.7
Notting Hill | Genres: 'Romance','Comedy','Drama' | Rating: 7.0
Molly | Genres: 'Drama','Comedy','Romance' | Rating: 5.5
Lars and the Real Girl | Genres: 'Comedy','Romance','Drama' | Rating: 7.1
The Devil Wears Prada | Genres: 'Comedy','Drama','Romance' | Rating: 7.0
Larry Crowne | Genres: 'Comedy','Romance','Drama' | Rating: 5.7
Forrest Gump | Genres: 'Comedy','Drama','Romance' | Rating: 8.2


The predicted rating for Love Actually is: 6.740000
The actual rating for Love Actually is 7.000000


As shown in the examples , given a name of a movie - a list of 10 similar movies is generated and its rating is predicted using the 10 nearest neighbours.
So yeah, input your favorite movies and get recommendations.
Cheers!